### 1. Подготовка

Чтобы подготовиться к сбору данных о товарах из [онлайн-магазина Ленты](https://lenta.com/), импортируем необходимые библиотеки и задаём ключевые параметры для работы с API: базовый URL для получения категорий, текущую дату для отслеживания и заголовки запроса. Словарь HEADERS содержит минимальный набор заголовков, необходимых для получения корректного ответа от API Ленты. Этот набор был определён экспериментально.

In [1]:
import requests
import json
import time
import random
import pandas as pd
from datetime import date

# задаём базовый URL и заголовки для запросов к сайту
CATEGORIES_URL = "https://lenta.com/api-gateway/v1/catalog/categories?timestamp="
CHECK_DATE = date.today() # текущая дата для маркировки файлов

HEADERS = {
    'DeviceID': '42b24bb8-650a-f3a7-fa8d-593e40b478d4',
    'Experiments': ('exp_recommendation_cms.true, exp_apigw_purchase.test, exp_lentapay.test, exp_omni_price.test, '
                    'exp_profile_bell.test, exp_newui_cancel_order.test, exp_newui_history_active_action.test_stars, '
                    'exp_comment_picker_and_courier.test, exp_general_editing_page.test, exp_cl_omni_support.test, '
                    'exp_cl_omni_authorization.test, exp_onboarding_sbp.default, exp_fullscreen.test, exp_profile_login.false, '
                    'exp_new_notifications_show_unauthorized.test, exp_assembly_cost_location.cart, exp_search_bottom.default, '
                    'exp_onboarding_editing_order.test, exp_cart_new_carousel.default, exp_newui_cart_cancel_editing.test, '
                    'exp_newui_cart_button.test, exp_new_promov3., exp_sbp_enabled.test, exp_new_my_goods.test, exp_ui_catalog.test, '
                    'exp_search_out_of_stock.default, exp_profile_settings_email.default, exp_cl_omni_refusalprintreceipts.test, '
                    'exp_cl_omni_refusalprintcoupons.test, exp_accrual_history.test, exp_personal_recommendations.control, '
                    'exp_newui_chips.test, exp_loyalty_categories.test, exp_growthbooks_aa.OFF, exp_test_ch_web.def, '
                    'exp_search_suggestions_popular_sku.default, exp_cancel_subscription.test_2, exp_manage_subscription.control, '
                    'exp_cl_new_csi.default, exp_cl_new_csat.default, exp_delivery_price_info.default, exp_personal_promo_navigation.test, '
                    'exp_web_feature_test.true, exp_interval_jump.default, exp_cardOne_promo_type.test, exp_qr_cnc.test, '
                    'exp_popup_about_order.test, exp_apigw_recommendations.test, exp_where_place_cnc.control, '
                    'exp_editing_cnc_onboarding.default, exp_editing_cnc.default, exp_selection_carousel.test, '
                    'exp_pickup_in_delivery.false, exp_feature_kpp_test.false, exp_welcome_onboarding.default, exp_cl_new_splash.default, '
                    'exp_web_referral_program_type.default, exp_where_place_new.default, exp_start_page.default, '
                    'exp_promocode_bd_coupon.default, exp_personal_promo_swipe_animation.default, exp_default_payment_type.default, '
                    'exp_main_page_carousel_vs_banner.default, exp_start_page_onboarding.default, exp_newui_cart_check_edit.default, '
                    'exp_search_new_logic.default, exp_search_ds_pers_similar.default, exp_growthbooks_aa_id_based_feature.control, '
                    'exp_referral_program_type.default, exp_my_choice_search.default, exp_items_by_rating.default, '
                    'exp_can_accept_early.default, exp_test_gb_value.false, exp_online_subscription.default, exp_new_nps_keyboard.test, '
                    'exp_main_page_carousel_vs_banner_shop.default, exp_bathcing.default, exp_web_qr_cnc.default, '
                    'exp_hide_cash_payment_for_cnc_wo_adult_items.default, exp_web_promocode_bd_coupon.default, '
                    'exp_prices_per_quantum.default, exp_test.default123, exp_web_partner_coupons_separately.default, '
                    'exp_web_chips_online.default'),
    'SessionToken': 'E05BD3D1115FF4A5D5F0BC02D141EB5A',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36',
    'X-Delivery-Mode': 'pickup',
    'X-Platform': 'omniweb',
    'X-Retail-Brand': 'lo',
}

### 2. Веб-скрейпинг

Чтобы извлечь данные о товарах с сайта Ленты, мы действуем поэтапно: загружаем список категорий, собираем данные по каждой категории, очищаем результаты и сохраняем всё в структурированном формате.

#### 2.1. Получение категорий

Сначала мы получаем список товарных категорий через API Ленты. Функция извлекает нужные поля — *id*, *name*, родительскую категорию, *slug* (используется для формирования URL) и *level*, — и возвращает очищенный список.

Одним из обязательных параметров запроса является метка текущего времени в миллисекундах, она добавляется к URL запроса. Каждая категория имеет числовой идентификатор и slug; оба параметра необходимы для последующих запросов и построения URL.

In [ ]:
def fetch_categories():

    # получаем данные о категориях от API (не забываем метку времени)
    response = requests.get(f'{CATEGORIES_URL}{int(time.time() * 1000)}', headers=HEADERS)
    raw_categories = json.loads(response.text) # преобразуем JSON-ответ в список словарей с категориями
    
    cleaned_categories = []
    
    # проходим по полученным данным и выбираем только нужные поля
    for category in raw_categories["categories"]:    # выбираем категории верхнего уровня, сохраняем их параметры
        cleaned_categories.append({
            "id": category["id"], # идентификатор категории для последующего получения товаров
            "name": category["name"], # название — для справки
            "parent_id": category["parentId"] if category["parentId"] != 0 else None,        # идентификатор и название родительской категории
            "parent_name": category["parentName"] if category["parentName"] != '' else None, # пусто, если это категория верхенго уровня
            "slug": category["slug"], # имя категории, использумое в URL, нужно для получения товаров
            "level": category["level"] # уровень категории, помогает избежать повторного запроса тех же товаров
        })

    return cleaned_categories

Мы загружаем все доступные категории и сохраняем их в CSV-файл, чтобы данные было удобно просматривать и повторно использовать. Затем отбираем только категории второго уровня как компромисс по скорости и частоте запросов. С одной стороны, таких категорий меньше, чем третьего уровня (это ускоряет процесс и снижает риск превышения лимитов запросов), с другой — они более мелкие, чем категории верхнего уровня, что помогает сократить потери данных при ошибке запроса или блокировке.

In [ ]:
categories = fetch_categories()

categories_df = pd.DataFrame(categories)
categories_df.to_csv(f'categories-{CHECK_DATE}-lenta.csv', index=False)

In [ ]:
# необязательный промежуточный вывод списка категорий
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(categories_df)

In [ ]:
categories_2 = [cat for cat in categories if cat['level'] == 2] # выбираем только категории второго уровня
# categories_2 = categories_2[:] # эту строку можно использовать, чтобы продолжить сбор данных с определённого места в случае ошибки
total_categories_count = len(categories_2)

#### 2.2. Получение исходных данных о товарах

Мы получаем списки товаров, имитируя работу API сайта, который загружает товары порциями до 40 товаров за запрос (пагинация), смещая индекс (*offset*) при каждом следующем запросе. В отличие от некоторых других сайтов, Lenta использует POST-запросы для получения данных о товарах («items»), а не GET-запросы. В теле запроса передаются идентификатор категории, фиксированный лимит в 40 товаров и индекс смещения, увеличивающийся на 40 с каждой страницей. В запросе есть еще пара дополнительных полей (фильтры и сортировка), но они остаются неизменными.

В запросе также используется заголовок `Referer`, указывающий на страницу категории (сформированную из базового URL, slug и идентификатора), чтобы лучше имитировать поведение браузера — хотя это, по-видимому, не строго обязательно.

Мы перебираем все страницы внутри категории, пока не соберём все товары. Случайные паузы между запросами снижают риск блокировок и превышения лимитов.

В итоге получаем сырые данные о товарах в формате JSON, соответствующем структуре, используемой на сайте.

In [ ]:
def fetch_products(category):
    
    total_products = 1 # начальное значение, чтобы войти в цикл
    offset = 0
    products = []

    # задаём заголовок Referer, соответствующий запрашиваемой категории (может помочь избежать блокировки)
    HEADERS['Referer'] = f'https://lenta.com/catalog/{category["slug"]}-{category["id"]}/'
    url = 'https://lenta.com/api-gateway/v1/catalog/items'
        
    while offset < total_products:
        
        time.sleep(random.uniform(1, 5)) # случайная задержка, чтобы не вызвать блокировку

        # формируем тело POST-запроса
        json_data = {
            'categoryId': category['id'],
            'limit': 40,
            'offset': offset,
            'sort': {
                'type': 'popular',
                'order': 'desc',
            },
            'filters': {
                'range': [],
                'checkbox': [],
                'multicheckbox': [],
            },
        }
    
        response = requests.post(url, headers=HEADERS, json=json_data) # получаем данные товаров для текущей категории и страницы (смещения)
        response_data = json.loads(response.text)
        
        products += response_data["items"] # добавляем товары из ответа в общий список
        total_products = response_data["total"] # обновляем общее число товаров
        
        offset += 40 # переходим к следующей странице результатов
        
        print(f'{len(products)}..', end='') # индикатор прогресса: сколько товаров уже загружено

    return products    

#### 2.3. Очистка данных о товарах

Далее мы определяем функцию, которая извлекает из сырых данных только необходимые поля:

- Идентификатор категории (для справки)
- Название товара
- Обычная цена (преобразованная из копеек в рубли)
- Уточнение по цене (например, вес нетто)
- Название супермаркета

В итоге получается очищенный список товаров, готовый к сохранению или дальнейшему анализу.

In [ ]:
def clean_product_data(category, raw_products):

    cleaned_products = []

    # проходим по сырым данным и выбираем только нужные поля
    for product in raw_products:
        cleaned_products.append({
            "category_id": category["id"],
            "name": product["name"],
            "price": product["prices"]["priceRegular"] / 100, # преобразуем копейки в рубли
            "pricing_unit": product['weight']["package"], # уточняет за какую единицу указана цена или вес нетто
            "supermarket": 'Lenta' # название супермаркета как источника данных
        })

    return cleaned_products

#### 2.4. Сводим все вместе

В следующей ячейке кода:

1. Проходим по выбранным категориям второго уровня,
2. Получаем и очищаем данные по товарам каждой категории,
3. Добавляем очищенные данные в общий CSV-файл, собирая полный набор данных.

Для удобства отслеживания в процессе выводится количество загруженных товаров по каждой категории. Между итерациями добавляется небольшая случайная задержка, чтобы избежать ограничения по количеству запросов. Имя выходного файла содержит дату (`CHECK_DATE`), чтобы зафиксировать, когда данные были выгружены.

In [ ]:
fetched_categories_count = 0    # счётчик категорий для отслеживания прогресса загрузки

# создаём файл с заголовком таблицы для записи данных
products_df = pd.DataFrame(columns=["category_id", "name", "price", "pricing_unit", "supermarket"])
products_df.to_csv(f'scraped_products-{CHECK_DATE}-lenta.csv', index=False, mode='w')

for category in categories_2:
    
    raw_products = fetch_products(category) # загружаем товары
    new_products = clean_product_data(category, raw_products) # извлекаем только необходимые данные и добавляем новые товары в список

    products_df = pd.DataFrame(new_products)
    products_df.to_csv(f'scraped_products-{CHECK_DATE}-lenta.csv', index=False, mode='a', header=False)

    fetched_categories_count += 1
    print(f'Категория {category["id"]} обработана, загружено {fetched_categories_count} из {total_categories_count} категорий')
    
    time.sleep(random.uniform(1, 5))

print(f'Загрузка завершена. Результаты сохранены в файл scraped_products-{CHECK_DATE}-lenta.csv')

### 3. Фильтрация и нормализация данных о товарах

После сбора и очистки исходных данных о продуктах мы переходим к фильтрации, чтобы включить только те товары, которые мы планируем сравнивать. Этот этап состоит из нескольких шагов:

#### 3.1. Первичная обработка

Начинаем с загрузки ранее сохранённых данных по продуктам и категориям, удаляем поле *category_id*, которое больше не нужно. Также удаляем точные дубли, которые могли появиться, если один и тот же товар был в нескольких категориях.

In [ ]:
import pandas as pd
import re

categories = pd.read_csv('categories-2025-03-04-lenta.csv')
products_original = pd.read_csv('scraped_products-2025-03-04-lenta-complete.csv')

products = products_original.drop(['category_id'], axis=1)    # удаляем поле category_id
products = products.drop_duplicates() # удаляем дубли

#### 3.2. Фильтрация товаров по типу

Чтобы выбрать из набора только релевантные для сравнения товары, мы создаём словарь с типами продуктов и соответствующими регулярными выражениями. Каждое выражение выбирает базовую форму продукта, исключая вариации, которые не входят в сферу нашего анализа.

> **Примечание:** приведенные здесь регулярные выражения не универсальны и подобраны под конкретный набор данных.

In [ ]:
product_regex_map = {
    'rice': r'^рис\b(?!.*овощ)',
    'bread': r'(^хлеб\b|^багет\b|^батон\b)(?!.*печеноч)',
    'chicken_fillet': r'^филе (кур|груд)(?!.*(копч|соус|запеч|бедр|индей|утен))',
    'pork_leg': r'^окорок (свин|из свин)(?!.*(копч|соус))',
    'egg': r'^яйцо курин',
    'cucumber': r'^огур(цы|ец)(?!.*(солен|маринован|ягод))',
    'carrot': r'^морковь(?!.*(корей|отвар))', # выбирает "морковь" и некоторые вариации,
                                              # но исключает неподходящие "морковь по-корейски" или "морковь отварная"
    'onion': r'^лук репч(?!.*суш)',
    'tomato': r'^томаты(?!.*(сок|очищ|маринован|вялен|солен|измельч|кус))',
    'cabbage': r'^капуста\b.*белокоч',
    'eggplant': r'^баклажаны,',
    'banana': r'^банан(?!.*(вял|суш|куб))', # выбирает "банан" или "бананы", но исключает "бананы вяленые" или "бананы сушеные"
    'orange': r'^апельсин(?!.*(сахар|куб))',
    'milk': r'^молоко(?!.*(сгущ|кокос|сух|топл|коз|обогащ|витам|лактоз|печен))',
    'yogurt': r'^йогурт\b(?!.*питье)',
    'condensed_milk': r'^молоко.*сгущ(?!.*(варен|кофе|какао))',
    'green_tea': r'^чай зел(?!.*порош)',
    'black_tea': r'^чай черн',
    'ground_coffee': r'^кофе(?!.*(капсул|раствор|фильтр)).*молот',
    'sugar': r'^сахар\b(?!.*(ванил|коричн))',
    'salt': r'^соль(?!.*(посуд|ванн|розов|чесн|прян|купан|мельн))',
    'sunflower_oil': r'^масло\b(?!.*(оливк|спрей|аром|вкус|добавл)).*подсолн',
    'water': r'^вода\b(?!.*(малин|клюкв|лимон|цитр)).*негаз',
    'buckwheat': r'(^крупа\b.*гречн|^гречка\b)(?!.*зел)',
    'spaghetti': r'^макароны\b.*спагетти', # выбирает "макароны" и среди них только "спагетти"
    'rice_noodles': r'(^лапша|^вермишель).*(рис|фунчоз)(?!.*соус)',
    'tofu': r'(^продукт)(?!.*(копч|папр)).*тофу|^тофу(?!.*(гриб|томат))',
    'mango': r'^манго желт',
    'fish_sauce': r'^соус.*рыбн'
}
product_regex_list = '|'.join(product_regex_map.values()) # объединяем все регулярные выражения в одно, используя оператор OR (|)

# фильтруем товары, названия которых соответствуют одному из выбранных типов продуктов
filtered_products = products.loc[products.name.str.contains(product_regex_list, case=False, regex=True)]

#### 3.3. Присвоение типа продукта

Каждому отфильтрованному товару присваивается метка с типом продукта на основе совпадения по регулярному выражению.

In [ ]:
def assign_product_type(row):
    name = row['name']
    for product_type, regex in product_regex_map.items():
        match = re.search(regex, name, flags=re.IGNORECASE)
        if match:
            return product_type
    return None

filtered_products = filtered_products.copy()  # создаём копию, чтобы избежать предупреждений при добавлении новых полей через .loc
filtered_products.loc[:,'product_type'] = filtered_products.apply(assign_product_type, axis=1)

# необязательный промежуточный вывод списка отфильтрованных товаров
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(filtered_products)

#### 3.4. Извлечение и нормализация единиц измерения

Разные единицы товаров отличаются по количеству, весу или объёмы. Чтобы сравнение было корректным, мы извлекаем нужную информацию из названия товара или из поля с уточнением цены и рассчитываем нормализованные показатели — цену за килограмм, литр или за штуку.

- Вес в граммах
- Количество штук (в частности, яиц)
- Объём в миллилитрах

Каждое из значений извлекается по шаблону (через регулярное выражение). Не все товары содержат все эти значения, поэтому соответствующие колонки с нормализованными ценами (*price_kg*, *price_lit*, *price_unit*) могут остаться пустыми для некоторых товаров.

> **Примечание о весовых товарах:** изначально предполагалось, что для таких товаров цена указана за кг и, соответственно, вес равен 1 кг. Однако позже выяснилось, что это не так. Фактический вес передавался API в отдельном поле, но я не обратил на него внимание. Эта ошибка учтена позже на этапе финальной очистки данных.

In [ ]:
def extract_weight(row):
    """Извлекает общий вес в граммах из названия товара или уточнения цены.
    Поддерживает как одиночный вес, так и формат с несколькими порциями (например, '5x100г').
    Примечание: Для 'весовых' товаров вес принят равным 1 кг, это ошибка,
    которая будет исправлена позже на этапе финальной очистки данных.
    """
    
    name, pricing_unit = row['name'], row['pricing_unit']
    pricing_unit = str(pricing_unit) if pd.notna(pricing_unit) else "" # заменяем отсутствующие значения пустой строкой
    
    # формат с несколькими порциями (например, 5x100г)
    match = re.search(r'(\d+)(x|х)(\d+|\d+[,.]\d+)\s?г\b', name) # выбирает (число) х (число) г
    if match:
        portion = float(match.group(1).replace(',', '.')) # извлекаем количество порций
        per_portion = float(match.group(3).replace(',', '.')) # извлекаем вес одной порции
        return portion * per_portion # возвращаем общий вес
    # одиночный вес (в граммах или килограммах)
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(г\b|кг)', name)
    if match:
        weight = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return weight * 1000 if unit == 'кг' else weight # преобразуем килограммы в граммы
    # если в name ничего не найдено — проверяем pricing_unit
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(г\b|кг)', pricing_unit)
    if match:
        weight = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return weight * 1000 if unit == 'кг' else weight
    # если в названии есть слово "весовой", значит указана цена за 1 кг
    match = re.search(r'\bвесов', name)
    if match:
        weight = 1000
        return weight

    return None  # если ничего не найдено

# следующие две функции работают по той же логике, что и extract_weight, но для штук и миллилитров
def extract_number_of_units(row):
    """Извлекает количество единиц из названия товара или уточнения цены
    ('шт' - штуки; 'пак', 'пир', 'саш' - для чая: пакетики, пирамидки, саше).
    """
    
    name, pricing_unit = row['name'], row['pricing_unit']
    pricing_unit = str(pricing_unit) if pd.notna(pricing_unit) else ""

    # проверяем название
    match = re.search(r'(\d+)\s?(шт|пак|пир|саш)', name)
    if match:
        number_of_units = int(match.group(1))
        return number_of_units
    # проверяем pricing_unit
    match = re.search(r'(\d+)\s?(шт|пак|пир|саш)', pricing_unit)
    if match:
        number_of_units = int(match.group(1))
        return number_of_units

    return None  # если ничего не найдено

def extract_volume(row):
    """Извлекает общий объем в миллилитрах из названия товара или уточнения цены.
    Поддерживает как одиночный объем, так и групповую упаковку (например, '5x100мл').
    """
    
    name, pricing_unit = row['name'], row['pricing_unit']
    pricing_unit = str(pricing_unit) if pd.notna(pricing_unit) else ""

    # формат с несколькими порциями (например, 5x100мл)
    match = re.search(r'(\d+)(x|х)(\d+|\d+[,.]\d+)\s?мл', name)
    if match:
        portion = float(match.group(1).replace(',', '.'))
        per_portion = float(match.group(3).replace(',', '.'))
        return portion * per_portion
    # одиночный объем (в литрах или миллилитрах)
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(мл|л\b)', name)
    if match:
        volume = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return volume * 1000 if unit == 'л' else volume
    # проверяем pricing_unit
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(мл|л\b)', pricing_unit)
    if match:
        volume = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return volume * 1000 if unit == 'л' else volume

    return None  # если ничего не найдено

filtered_products = filtered_products.copy()  # пересоздаем датафрейм

# вычисляем нормализованные цены
filtered_products.loc[:,'weight'] = filtered_products.apply(extract_weight, axis=1)  # столбец с весом в граммах
filtered_products.loc[:,'price_kg'] = filtered_products.price / filtered_products.weight * 1000   # столбец с ценой за кг

filtered_products.loc[:,'number_of_units'] = filtered_products.apply(extract_number_of_units, axis=1)  # столбец с количеством штук
filtered_products.loc[:,'price_unit'] = filtered_products.price / filtered_products.number_of_units   # столбец с ценой за штуку

filtered_products.loc[:,'volume'] = filtered_products.apply(extract_volume, axis=1)  # столбец с объемом в мл
filtered_products.loc[:,'price_lit'] = filtered_products.price / filtered_products.volume * 1000   # столбец с ценой за литр

# необязательный промежуточный вывод
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(filtered_products)

#### 3.5. Сохранение финального отфильтрованного набора данных

Наконец, отфильтрованный и дополненный набор данных сохраняется в новый CSV-файл для дальнейшего анализа.

In [ ]:
filtered_products.to_csv(f'filtered_products-2025-03-04-lenta.csv')